# Глава 9 Снижение размерности с помощью выделения признаков

Не все признаки создаются равными, и выделение признаков для снижения размерности имеет совершенно конкретную цель - преобразование нашего набора признаков р(исх), чтобы в коненом итоге прийти к новому набору р(нов), где р(исх)>р(нов), сохраняя при этом подавляющуючасть исходной информации.

Мы уменьшим количество признаков с небольшой потерей способности наших данных генерировать высококачественные предсказания.

Но новые признаки, которые мы генерируем, не поддаются интерпретированию людьми. Они будут содержать почти столько же возможностей для тренировки моделей, но для человеческого глаза будет выглядеть как набор случайных чисел.

Если хотим сохранить способность интерпретировать модели, лучше применить другой метод снижения размерности - уменьшения размерности посредством отбора признаков.

# Снижение признаков с помощью главных компонент

Дан набор признаков, и требуется сократить количество признаков, сохраняя при этом дисперсию данных.

Используем анализ главных компонент с помощью класса PCA библиотеки scikit-learn.

In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import datasets

In [2]:
# загрузить данные
digits = datasets.load_digits()

In [4]:
print(list(digits.keys()))

['data', 'target', 'target_names', 'images', 'DESCR']


In [5]:
# стандартизировать матрицу признаков
features = StandardScaler().fit_transform(digits.data)

In [6]:
# создать объект PCA, который сохранит 99% дисперсии
pca = PCA(n_components=0.99, whiten=True)

In [7]:
features_pca = pca.fit_transform(features)

In [8]:
print('Исходное количество признаков: ', features.shape[1])
print('Сокращенное количество признаков: ', features_pca.shape[1])

Исходное количество признаков:  64
Сокращенное количество признаков:  54


Метод PCA проецирует наблюдения на главные компоненты матрицы признаков, которые сохраняют наибольшую дисперсию. PCA - является неконтролируемым методом(без учителя) - он не использует информацию из вектора целей и вместо этого рассматривает только матрицу признаков.

У аргумента n_components есть две операции в зависимости от заданного значения, если значение этого аргумента больше 1, то n_components вернет указанное значение признаков. Если n_components находится между 0 и 1, то созданный объект возвращает минимальное количество признаков, которые сохраняют заданную дисперсию. Обычно используют 0.99 и 0.95.

whiten=True   

преобразует значение каждой главной компоненты, чтобы они имели нулевое среднее и единичную дисперсию.

svd_solver = "randomized"  - реализует стохастический алгоритм нахождения первых главных компонент(занимает меньше времени)

Метод PCA позволяет уменьшить размерность на 10 признаков, сохраняя 99% информации.

# Уменьшение количества признаков, когда данные линейно неразделимы

Возможно ваши данные линейно неразделимы, и требуется сократить размерности

Применить расширение анализа главных компонент, в котором используются ядра для нелинейного уменьшения размерности:

In [1]:
#Загрузить библиотеки
from sklearn.decomposition import PCA, KernelPCA
from sklearn.datasets import make_circles

In [2]:
#Создать линейно-неразделимые данные
features, _ = make_circles(n_samples=1000, random_state=1, noise=0.1, factor=0.1)

In [3]:
#Применить ядерный PCA
#с радиально-базисным функциональным ядром(RBF-ядром)
kpca = KernelPCA(kernel="rbf", gamma=15, n_components=1)
features_kpca = kpca.fit_transform(features)

In [4]:
print('Исходное количество признаков: ', features.shape[1])
print('Сокращенное количество признаков: ', features_kpca.shape[1])

Исходное количество признаков:  2
Сокращенное количество признаков:  1


Если данные линейно разделимы, то PCA работает хорошо. Если данные не являются линейно-разделимыми, то линейное преобразование работать не будет.

Функция make_circles создает линейно-неразделимые данные (один класс окружен со всех сторон другим классом).

Если бы для уменьшения размерностей наших данных мы использовали линейный метод PCA, то два класса были бы линейно спроецированы на первую главную компоненту таким образом, что они стали бы переплетенными.

Мы хотели бы иметь преобразование, которое сокращало бы размерности, а также делало бы данные линейно-разделимыми.

Это может делать ядерный PCA.

Ядра позволяют проецировать линейно неразделимые данные в более высокую размерность, где они линейно разделимы. Этот подход называется ядерным трюком.

В объекте, созданным из класса KernelPCA библиотеки scikit-learn можно использовать несколько ядер, задаваемых с помощью параметра kernel.

Виды ядер:

1)гауссово радиально-базисное функциональное ядро rbf

2)полиноминальное ядро poly

3)сигмоидное ядро sigmoid

4)линейная проекция linear

В ядерном PCA мы не можем указать n_components как дисперсию. Мы должны задать ряд параметров. Плюс есть еще гиперпараметры(радиально-базисная функция требует значения gamma).

# Уменьшение количества признаков путем максимизации разделимости классов

Требуется сократить признаки, используемые классификатором

Попробовать линейный, дискриминантный анализ (linear discriminant analysis, LDA), чтобы спроецировать объекты на оси компонент, которые максимизируют разделение классов.

Дискриминантный анализ — это статистический метод, предназначенный для изучения отличий между двумя или большим количеством групп объектов с использованием данных о разнообразии нескольких признаков, отличающих эти объекты друг от друга.

https://ru.wikipedia.org/wiki/%D0%94%D0%B8%D1%81%D0%BA%D1%80%D0%B8%D0%BC%D0%B8%D0%BD%D0%B0%D0%BD%D1%82%D0%BD%D1%8B%D0%B9_%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7

In [6]:
from sklearn import datasets
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [7]:
#загрузить набор данных цветков ириса
iris = datasets.load_iris()
features = iris.data
target = iris.target

In [8]:
#создать объект и выполнить LDA, затем использовать
#его для преобразования признаков
lda = LinearDiscriminantAnalysis(n_components=1)
features_lda = lda.fit(features, target).transform(features)

In [9]:
print('Исходное количество признаков: ', features.shape[1])
print('Сокращенное количество признаков: ', features_lda.shape[1])

Исходное количество признаков:  4
Сокращенное количество признаков:  1


In [ ]:
# атрибут explained_variance_ratio_    - просмотр объема дисперсии (объясненной каждой компонентой)

In [10]:
lda.explained_variance_ratio_

array([0.9912126])

Метод LDA работает аналогично анализу главных компонент(PCA): он проецирует пространство признаков на пространство более низкой размерности. Однако в PCA нас интересовали только те оси компонент, которые максимизируют дисперсию данных, в то время как в LDA есть дополнительная цель - максимизировать различия между классами. 

n_components - указывает на количество признаков, которые требуется вернуть.

Чтобы выяснить какое знаение использовать в n_components: можно воспользоваться коэффициентом  explained_variance_ratio_ , он сообщает нам дисперсию, объясняемую каждым выводимым признаком.

Можно выполнить линейный дискриминантный анализ на основе класса LinearDiscriminantAnalysis с n_components=None, чтобы вернуть коэффициент дисперсии, объясненный каждой компонентой, а затем вычислить, сколько требуется компонент, чтобы превысить некий порог дисперсии.

In [18]:
#создать объект и выполнить LDA
lda = LinearDiscriminantAnalysis(n_components=None)
features_lda = lda.fit(features, target)

In [19]:
#создать массив коэффициентов объясненной дисперсии
lda_var_ratios = lda.explained_variance_ratio_
print(lda_var_ratios)

[0.9912126 0.0087874]


In [13]:
#создать функцию
def select_n_components(var_ratio, goal_val: float) -> int:
    #задать исходную объясненную на данный момент дисперсию
    total_variance = 0.0
    
    #задать исходное количество признаков
    n_components = 0
    
    #для объясненной дисперсии каждого признака:
    for explained_variance in var_ratio:
        #добавить объясненную дисперсию к итогу
        total_variance += explained_variance
        
        #добавить единицу к количеству компонент
        n_components += 1
        
        #если достигнут целевой уровень объясненной дисперсии
        if total_variance >= goal_val:
            #завершить цикл
            break
    #вернуть количество компонент       
    return n_components

In [14]:
#выполнить функцию
select_n_components(lda_var_ratios, 0.95)

1

# Уменьшение количества признаков с использованием разложения матрицы

https://ru.wikipedia.org/wiki/%D0%9D%D0%B5%D0%BE%D1%82%D1%80%D0%B8%D1%86%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D0%BE%D0%B5_%D0%BC%D0%B0%D1%82%D1%80%D0%B8%D1%87%D0%BD%D0%BE%D0%B5_%D1%80%D0%B0%D0%B7%D0%BB%D0%BE%D0%B6%D0%B5%D0%BD%D0%B8%D0%B5

Дана матрица признаков с неотрицательными значениями и требуется уменьшить ее размерность.

Использовать разложение неотрицательной матрицы с целью уменьшения размерности матрицы признаков:

In [ ]:
#NMF (non-negative matrix factorization)

In [1]:
#загрузить библиотеки
from sklearn.decomposition import NMF
from sklearn import datasets

In [2]:
#загрузить данные
digits = datasets.load_digits()

In [3]:
#загрузить  матрицу признаков
features = digits.data

In [4]:
#создать NMF и выполнить его подгонку
nmf = NMF(n_components=10, random_state=1)
features_nmf = nmf.fit_transform(features)

In [5]:
#показать результаты
print('Исходное количество признаков: ', features.shape[1])
print('Сокращенное количество признаков: ', features_nmf.shape[1])

Исходное количество признаков:  64
Сокращенное количество признаков:  10


Разложение неотрицательной матрицы NMF является неконтролируемым методом уменьшения линейной размерности, который факторизует(разбивает на несолько матриц, произведение которых соответствует исходной матрице) - матрицу признаков в матрицы, представляющие скрытую связь между наблюдениями и их признаками.

Метод NMF может сократить размерность, поскольку в матричном умножении два сомножителя(умножаемые матрицы)  - могут иметь значительно меньшие размерности, чем матрица произведения.

Если дано желаемое число возвращаемых признаков - r, то метод NMF  разлагает матрицу признаков: V = W x H

V - наша матрица признаков d x n

W - матрица в d x r

H - матрица r x n

Скорректировав r - можно задать объем желаемого уменьшения размерности

Требование: матрица не может содержать отрицательные значения.

Метод NMF  не предоставляет объясненную дисперсию результирующих признаков.

То лучший способ найти оптимальное значение n_components - попытаться найти в диапазоне значений то, которое дает наилучший результат.

# Уменьшение количества признаков на разряженных данных

https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD

Дана разряженная матриа признаков, и требуется уменьшить ее размерность.

Использовать усеченное сингулярное разложение TSVD (truncated singular value decomposition)

In [6]:
#загрузить библиотеки
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
from sklearn import datasets
import numpy as np

In [7]:
#загрузить данные
digits = datasets.load_digits()

In [9]:
#стандартизировать матрицу признаков
features = StandardScaler().fit_transform(digits.data)

In [10]:
#сделать разряженную матрицу
features_sparse = csr_matrix(features)

In [12]:
#сделать объект TSVD
tsvd = TruncatedSVD(n_components=10)

In [13]:
#выполнить TSVD на разряженной матрице
features_sparse_tsvd = tsvd.fit(features_sparse).transform(features_sparse)

In [14]:
#показать результаты
print('Исходное количество признаков: ', features_sparse.shape[1])
print('Сокращенное количество признаков: ', features_sparse_tsvd.shape[1])

Исходное количество признаков:  64
Сокращенное количество признаков:  10


Усеченное сингулярное разложение TSVD похоже на анализ главных компонент PCA (PCA на одном из своих шагов часто используется SVD (неусеченное сингулярное разложение).

В обычном сингулярном разложении при наличии d признаков создаются матрицы сомножители размера dxd, тогда как усеченное сингулярное разложение вернет сомножители, которые имеют размер n x n, где n - предварительно заданы параметром.

TSVD в отличии от  PCA работает на разряженных матрицах признаков.

Одна из проблем (TSVD) - в зависимости от того, как этот метод использует генератор случайных чисел, знаки результата могут меняться от подгонки к подгонке.

Простое решение использовать метод fit в конвейре предобработки - 1 раз, затем несколько раз transform

Как и в случае линейного дискриминантного анализа, мы должны указать количество признаков (компонент), которые  мы хотим вывести. Это делается с помощью параметра n_components.

Но каково оптимальное количество компонент?

Одна стратегия состоит в том, чтобы включить n_components как гиперпараметр для оптимизации при отборе модели( т е выбрать значение для n_components, которое производит наилучшую натренированную модель).

Альтернатива: TSVD - предоставляет нам коэффициент объясненной дисперсии каждой компоненты исходной матрицы признаков, мы можем выделить ряд компонент, которые объясняют желаемый набор дисперсии( общепринятьо использовать 95% и 99%).

In [15]:
#суммировать коэффициенты объясненной дисперсии первых 3-х компонент
tsvd.explained_variance_ratio_[0:3].sum()

0.3003938538258395

In [ ]:
#это значит, первые три компоненты - объясняют 30% дисперсии

этот процесс можно автоматизировать

In [17]:
#создать и выполнить TSVD с числом признаков меньше на 1
tsvd = TruncatedSVD(n_components=features_sparse.shape[1]-1)
features_tsvd = tsvd.fit(features)

In [18]:
#поместить в список объясненные дисперсии
tsvd_var_ratios = tsvd.explained_variance_ratio_

In [19]:
#создать функцию
def select_n_components(var_ratio, goal_val):
    #задать исходную объясненную на данный момент дисперсию
    total_variance = 0.0
    
    #задать исходное количество признаков
    n_components = 0
    
    #для объясненной дисперсии каждого признака:
    for explained_variance in var_ratio:
        #добавить объясненную дисперсию к итогу
        total_variance += explained_variance
        
        #добавить единицу к количеству компонент
        n_components += 1
        
        #если достигнут целевой уровень объясненной дисперсии
        if total_variance >= goal_val:
            #завершить цикл
            break
    #вернуть количество компонент       
    return n_components

In [20]:
select_n_components(tsvd_var_ratios, 0.95)

40